# Use TensorFlow 1.x.

In [0]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

# Install gdown Python package.

In [0]:
!pip install -U --no-cache-dir gdown

# Install AttGAN dependencies.

In [0]:
!pip install opencv-python scikit-image tqdm oyaml

# Clone AttGAN git repository.

### Clone AttGAN git repository.

In [0]:
!git clone https://github.com/LynnHo/AttGAN-Tensorflow.git

### See cloned repository.

In [0]:
!ls -al
!pwd

### Set the current root directory.

In [0]:
import os

dataset_dir = '/content/AttGAN-Tensorflow'
os.chdir(dataset_dir)

!ls -al

# Download AttGAN model weights.

### Download AttGAN model weights.

In [0]:
!gdown --id 1Oy4F1xtYdxj4iyiLyaEd-dkGIJ0mwo41

### See downloaded weights.

In [0]:
!ls -al

### Extract model weights at appropiate location. 

In [0]:
!mkdir output
!mv AttGAN_128.zip output/.
!unzip output/AttGAN_128.zip -d output/

### See extracted model weights.

In [0]:
!ls -al output
!ls -al output/AttGAN_128

# Download CelebA dataset.

### Download CelebA dataset.
*   Download the dataset from Google drive. 
OR
*   Download the dataset from Udacity hosted datasets.



In [0]:
!gdown --id 0B7EVK8r0v71pZjFTYXZWM3FlRnM

In [0]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

### See downloaded CelebA dataset.

In [0]:
!ls -al

### Extract CelebA dataset at appropiate location.

In [0]:
!mkdir -p data/img_celeba
!mv celeba.zip data/img_celeba/.
!unzip ./data/img_celeba/celeba.zip -d ./data/img_celeba/

### See extracted CelebA dataset.

In [0]:
!mv data/img_celeba/img_align_celeba data/img_celeba/img_celeba
!ls -al ./data/img_celeba/

# Download CelebA dataset annotations.

### Download dataset annotations.

In [0]:
!gdown https://drive.google.com/uc?id=1xd-d1WRnbt3yJnwh5ORGZI3g-YS-fKM9

### Extract dataset annotations at appropiate location.

In [0]:
!mkdir -p data/img_celeba
!mv annotations.zip data/img_celeba/.
!unzip ./data/img_celeba/annotations.zip -d ./data/img_celeba/

# Align images.

In [0]:
!python ./scripts/align.py

In [0]:
!ls -al data/img_celeba

# Test application.

In [0]:
!python test_slide.py --experiment_name AttGAN_128 --test_att_name Eyeglasses